## Multi Variants Endpoint (ABTest)

라이브러리

In [1]:
# 데이터 처리 및 분석
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', None)

# 머신러닝
from sklearn.model_selection import train_test_split

# AWS 관련
import sagemaker
from sagemaker.utils import name_from_base
import boto3
import awswrangler as wr

# 기타 유틸리티
import requests
import zipfile
import pickle
import os
import json
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

### 데이터 전처리

데이터 다운로드

In [7]:
# URL에서 ZIP 파일 다운로드
url = "https://openpowerlifting.gitlab.io/opl-csv/files/openpowerlifting-latest.zip"
response = requests.get(url)

# dataset 폴더 생성 (없는 경우)
os.makedirs('dataset', exist_ok=True)
# ZIP 파일 저장
zip_path = os.path.join('dataset', 'openpowerlifting-latest.zip')
with open(zip_path, 'wb') as f:
    f.write(response.content)

In [8]:
# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')

# ZIP 파일 제거
os.remove(zip_path)

print("파일 다운로드, 압축 해제 및 ZIP 파일 제거가 완료되었습니다.")

파일 다운로드, 압축 해제 및 ZIP 파일 제거가 완료되었습니다.


In [9]:
# URL에서 ZIP 파일 다운로드
url = "https://openpowerlifting.gitlab.io/opl-csv/files/openipf-latest.zip"
response = requests.get(url)

# ZIP 파일 저장
zip_path = os.path.join('dataset', 'openipf-latest.zip')
with open(zip_path, 'wb') as f:
    f.write(response.content)

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')

# ZIP 파일 제거
os.remove(zip_path)

print("파일 다운로드, 압축 해제 및 ZIP 파일 제거가 완료되었습니다.")


파일 다운로드, 압축 해제 및 ZIP 파일 제거가 완료되었습니다.


데이터 로드

In [10]:
from glob import glob

filename = glob("dataset/openpower*/*.csv")[0]
filename

'dataset/openpowerlifting-2024-08-24/openpowerlifting-2024-08-24-9e0647b3.csv'

In [48]:
df = pd.read_csv(filename)
df.head()

/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/ipykernel_15061/1403853012.py:1: DtypeWarning: Columns (31,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Bench4Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Place,Dots,Wilks,Glossbrenner,Goodlift,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName,Sanctioned
0,E.S. Denisenko,F,B,Raw,28.5,24-34,24-39,Open,67.3,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-45.0,-45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GSF-Belarus,NaN,2019-06-22,Belarus,NaN,Luninets,Bison Power Cup,Yes
1,I.S. Lebetskaya,F,B,Raw,43.5,40-44,40-49,Open,73.2,NaN,NaN,NaN,NaN,NaN,NaN,80.0,85.0,90.0,NaN,90.0,NaN,NaN,NaN,NaN,NaN,90.0,1,88.80,86.89,76.50,70.06,NaN,NaN,NaN,GSF-Belarus,NaN,2019-06-22,Belarus,NaN,Luninets,Bison Power Cup,Yes
2,K. Yakimovich,F,B,Raw,26.5,24-34,24-39,Open,60.6,NaN,NaN,NaN,NaN,NaN,NaN,40.0,42.5,45.0,NaN,45.0,NaN,NaN,NaN,NaN,NaN,45.0,2,49.57,49.79,43.98,38.42,NaN,NaN,NaN,GSF-Belarus,NaN,2019-06-22,Belarus,NaN,Luninets,Bison Power Cup,Yes
3,A.G. Golneva,F,B,Raw,19.5,20-23,19-23,Juniors 17-21,50.3,NaN,NaN,NaN,NaN,NaN,NaN,32.5,35.0,-37.5,NaN,35.0,NaN,NaN,NaN,NaN,NaN,35.0,2,43.67,44.76,39.73,34.55,NaN,NaN,NaN,GSF-Belarus,NaN,2019-06-22,Belarus,NaN,Luninets,Bison Power Cup,Yes
4,E.V. Marunevskaya,F,B,Raw,19.5,20-23,19-23,Juniors 17-21,63.7,NaN,NaN,NaN,NaN,NaN,NaN,40.0,42.5,-45.0,NaN,42.5,NaN,NaN,NaN,NaN,NaN,42.5,1,45.40,45.26,39.96,35.25,NaN,NaN,NaN,GSF-Belarus,NaN,2019-06-22,Belarus,NaN,Luninets,Bison Power Cup,Yes


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3268349 entries, 0 to 3268348
Data columns (total 42 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Name              object 
 1   Sex               object 
 2   Event             object 
 3   Equipment         object 
 4   Age               float64
 5   AgeClass          object 
 6   BirthYearClass    object 
 7   Division          object 
 8   BodyweightKg      float64
 9   WeightClassKg     object 
 10  Squat1Kg          float64
 11  Squat2Kg          float64
 12  Squat3Kg          float64
 13  Squat4Kg          float64
 14  Best3SquatKg      float64
 15  Bench1Kg          float64
 16  Bench2Kg          float64
 17  Bench3Kg          float64
 18  Bench4Kg          float64
 19  Best3BenchKg      float64
 20  Deadlift1Kg       float64
 21  Deadlift2Kg       float64
 22  Deadlift3Kg       float64
 23  Deadlift4Kg       float64
 24  Best3DeadliftKg   float64
 25  TotalKg           float64
 26  Place         

데이터 전처리

In [50]:
# 각 컬럼별 결측치 개수 출력
missing_values = df.isnull().sum()

print("각 컬럼별 결측치 개수:")
for column, count in missing_values.items():
    print(f"{column}: {count}")

각 컬럼별 결측치 개수:
Name: 0
Sex: 0
Event: 0
Equipment: 0
Age: 1176337
AgeClass: 894084
BirthYearClass: 1104524
Division: 1696
BodyweightKg: 40460
WeightClassKg: 44893
Squat1Kg: 2274026
Squat2Kg: 2283845
Squat3Kg: 2309173
Squat4Kg: 3259545
Best3SquatKg: 1050464
Bench1Kg: 1771832
Bench2Kg: 1788411
Bench3Kg: 1833139
Bench4Kg: 3246698
Best3BenchKg: 381902
Deadlift1Kg: 2150174
Deadlift2Kg: 2169234
Deadlift3Kg: 2211920
Deadlift4Kg: 3244287
Best3DeadliftKg: 871684
TotalKg: 219796
Place: 0
Dots: 245049
Wilks: 245049
Glossbrenner: 245049
Goodlift: 507057
Tested: 839174
Country: 1399176
State: 2575505
Federation: 0
ParentFederation: 1205090
Date: 0
MeetCountry: 0
MeetState: 908788
MeetTown: 454222
MeetName: 0
Sanctioned: 0


In [51]:
whole_data = df.copy()

In [52]:
# Tested 컬럼의 결측치 'No'로 치환
print(whole_data['Tested'].isnull().sum())
whole_data['Tested'] = whole_data[['Tested']].fillna('No')
whole_data['Tested'].value_counts()

839174


Tested
Yes    2429175
No      839174
Name: count, dtype: int64

In [53]:
# 장소 전처리
places = whole_data['Place'].unique()
print(places)
non_numeric_places = [val for val in places if not val.isnumeric() ]
whole_data['Place'] = whole_data['Place'].replace(non_numeric_places, -1).astype(int)
print(whole_data['Place'].unique())

['DQ' '1' '2' '3' '4' '5' '6' '10' '11' '12' '13' '9' '7' '8' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 'NS' 'DD' '31' '30' '32' 'G' '33' '34' '35' '37' '36' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '57' '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69'
 '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83'
 '84' '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97'
 '98' '99' '100' '101' '102' '103' '104' '105' '106' '107' '108' '109'
 '110' '111' '112' '113' '114' '115' '116' '117' '118' '119' '120' '195']
[ -1   1   2   3   4   5   6  10  11  12  13   9   7   8  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  31  30  32  33  34  35
  37  36  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86 

In [54]:
# BodyweightKg 열의 결측치를 중앙값으로 채웁니다.
whole_data["BodyweightKg"] = whole_data["BodyweightKg"].fillna(whole_data["BodyweightKg"].median())

In [55]:
# 체중을 카테고리로 매핑하는 함수를 정의합니다.
# 30kg 미만은 0 카테고리, 그 이상은 10kg 단위로 카테고리를 증가시킵니다.
def map_weight_to_category(weight):
    weight = float(weight)
    if weight < 30:
        return 0
    else:
        category = ((weight - 30) // 10) + 1
        return int(category)
    
whole_data.loc[:, 'KgWeightClass'] = whole_data['BodyweightKg'].apply(map_weight_to_category)

In [56]:
# 'Date' 열을 '-'로 분할하여 'Year', 'Month', 'Day' 열 생성
whole_data[['Year', 'Month', 'Day']] = whole_data['Date'].str.split('-', expand=True)

# 'Year', 'Month', 'Day' 열을 정수형으로 변환
whole_data[['Year', 'Month', 'Day']] = whole_data[['Year', 'Month', 'Day']].astype(int)

In [57]:
# 범주형 변수로 변환할 열들을 정의합니다.
columns_to_convert = ["Sex", "Event", "Equipment", "Tested", "Federation", 'ParentFederation', 'Sanctioned', 'State']
whole_data[columns_to_convert] = whole_data[columns_to_convert].astype('category')
whole_data[columns_to_convert].head(10)

,Sex,Event,Equipment,Tested,Federation,ParentFederation,Sanctioned,State
0,F,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
1,F,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
2,F,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
3,F,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
4,F,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
5,M,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
6,M,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
7,M,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
8,M,B,Raw,No,GSF-Belarus,NaN,Yes,NaN
9,M,B,Raw,No,GSF-Belarus,NaN,Yes,NaN


In [58]:
# sklearn의 LabelEncoder를 임포트합니다
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체를 생성합니다
encoder = LabelEncoder()

# columns_to_convert에 있는 각 열에 대해 반복합니다
for col in columns_to_convert:
    # 해당 열의 데이터를 숫자로 인코딩합니다
    whole_data[col] = encoder.fit_transform(whole_data[col])

In [59]:
# 정의된 열들에 대해 결측값을 0.0으로 채우고, 음수 값을 0으로 클리핑합니다
columns_to_fill = ['Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg'] 
whole_data[columns_to_fill] = whole_data[columns_to_fill].apply(lambda x: x.fillna(0.0).clip(lower=0))

In [60]:
# 각 리프팅 종목(스쿼트, 벤치프레스, 데드리프트)에 대해 최고 기록을 계산합니다.
whole_data['Best3SquatKg'] = whole_data[['Squat1Kg', 'Squat2Kg', 'Squat3Kg']].apply(max, axis=1)
whole_data['Best3BenchKg'] = whole_data[['Bench1Kg', 'Bench2Kg', 'Bench3Kg']].apply(max, axis=1)
whole_data['Best3DeadliftKg'] = whole_data[['Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg']].apply(max, axis=1)

In [61]:
# 각 리프팅 종목의 최고 기록을 합산하여 총 중량(TotalKg)을 계산합니다.
whole_data['TotalKg'] = whole_data['Best3SquatKg'] + whole_data['Best3BenchKg'] + whole_data['Best3DeadliftKg']

In [62]:
# 불필요한 열들을 제거합니다.
whole_data = whole_data.drop(columns = ["Name", "BirthYearClass", "AgeClass", "Division", "Squat4Kg", "Bench4Kg", "Deadlift4Kg",  "Glossbrenner", "MeetState", "Country", "WeightClassKg", "Date", 'State', 'MeetCountry', 'MeetTown', 'MeetName', 'Dots', 'Goodlift'], axis=1)

In [45]:
whole_data.head()

,Sex,Event,Equipment,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Dots,Wilks,Goodlift,Tested,Federation,ParentFederation,Sanctioned,KgWeightClass,Year,Month,Day
0,0,0,1,28.5,67.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,NaN,NaN,NaN,0,130,24,1,4,2019,6,22
1,0,0,1,43.5,73.2,0.0,0.0,0.0,0.0,80.0,85.0,90.0,90.0,0.0,0.0,0.0,0.0,90.0,1,88.80,86.89,70.06,0,130,24,1,5,2019,6,22
2,0,0,1,26.5,60.6,0.0,0.0,0.0,0.0,40.0,42.5,45.0,45.0,0.0,0.0,0.0,0.0,45.0,2,49.57,49.79,38.42,0,130,24,1,4,2019,6,22
3,0,0,1,19.5,50.3,0.0,0.0,0.0,0.0,32.5,35.0,0.0,35.0,0.0,0.0,0.0,0.0,35.0,2,43.67,44.76,34.55,0,130,24,1,3,2019,6,22
4,0,0,1,19.5,63.7,0.0,0.0,0.0,0.0,40.0,42.5,0.0,42.5,0.0,0.0,0.0,0.0,42.5,1,45.40,45.26,35.25,0,130,24,1,4,2019,6,22


In [63]:
# Wilks 열에서 0이 아닌 값의 개수를 계산합니다.
# 이는 Wilks 점수가 없는 (즉, 실격된) 선수의 수를 나타냅니다.
whole_data.Wilks.apply(lambda x : 0 if x else 1).sum()

0

In [67]:
# Wilks 값이 없는 경우를 실격으로 간주하여 'Disq' 열을 생성합니다
# Wilks 값이 없으면 1(실격), 있으면 0으로 설정합니다
whole_data['Disq'] = whole_data.Wilks.isna().astype(int)

# 'Wilks' 열을 제거합니다
# 이미 'Disq' 열로 정보를 추출했으므로 더 이상 필요하지 않습니다
whole_data.drop(columns=['Wilks'], inplace=True)

In [68]:
whole_data.to_csv('dataset/openpowerlifting_preperated.csv', index=False)

데이터 분리

In [71]:
# 특성(X)과 목표 변수(y)를 분리합니다
# 'Disq' 열을 제외한 모든 열을 특성으로 사용하고, 'Disq' 열을 목표 변수로 설정합니다
X = whole_data.drop('Disq', axis=1)  # 'Disq' 열을 제외한 모든 열을 특성으로 사용
y = whole_data['Disq']  # 'Disq' 열을 목표 변수로 설정

# 특성(X)과 목표 변수(y)의 형태를 출력하여 확인합니다
print(X.shape, y.shape)

(3268349, 27) (3268349,)


In [72]:
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.2, random_state = 2024)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.3, random_state = 2024)

In [73]:
print(X_train.shape, X_val.shape, X_test.shape)

(1830275, 27) (784404, 27) (653670, 27)


### 컨테이너 모델 훈련

SageMaker 세션 및 역할 설정

In [74]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

S3 데이터 저장 위치 설정

In [75]:
bucket_name = 'dante-sagemaker'
project_name = 'openpowerlift'

s3_training_file_location = r's3://{0}/{1}/input/{2}/'.format(bucket_name, project_name, 'training')
s3_validation_file_location =r's3://{0}/{1}/input/{2}/'.format(bucket_name, project_name, 'validation')

s3_xgb_output_location = r's3://{0}/{1}/output/{2}'.format(bucket_name, project_name, 'xgboost')
s3_xgb_checkpoint_location = r's3://{0}/{1}/checkpoint/{2}'.format(bucket_name, project_name, 'xgboost')

s3_lgb_output_location = r's3://{0}/{1}/output/{2}'.format(bucket_name, project_name, 'lightgbm')
s3_lgb_checkpoint_location = r's3://{0}/{1}/checkpoint/{2}'.format(bucket_name, project_name, 'lightgbm')

print('s3_training_file_location : ', s3_training_file_location)
print('s3_validation_file_location : ', s3_validation_file_location)
print('s3_xgb_output_location : ', s3_xgb_output_location)
print('s3_xgb_checkpoint_location : ', s3_xgb_checkpoint_location)
print('s3_lgb_output_location : ', s3_lgb_output_location)
print('s3_lgb_checkpoint_location : ', s3_lgb_checkpoint_location)

s3_training_file_location :  s3://dante-sagemaker/openpowerlift/input/training/
s3_validation_file_location :  s3://dante-sagemaker/openpowerlift/input/validation/
s3_xgb_output_location :  s3://dante-sagemaker/openpowerlift/output/xgboost
s3_xgb_checkpoint_location :  s3://dante-sagemaker/openpowerlift/checkpoint/xgboost
s3_lgb_output_location :  s3://dante-sagemaker/openpowerlift/output/lightgbm
s3_lgb_checkpoint_location :  s3://dante-sagemaker/openpowerlift/checkpoint/lightgbm


훈련/검증 데이터 업로드

In [78]:
df_train = pd.concat([y_train, X_train], axis=1)
df_train.head()

,Disq,Sex,Event,Equipment,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Tested,Federation,ParentFederation,Sanctioned,KgWeightClass,Year,Month,Day
2590320,0,1,5,1,19.5,121.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,246,24,1,10,2011,10,22
173502,0,1,0,2,30.5,110.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0,250.0,0.0,0.0,0.0,0.0,250.0,4,1,74,4,1,9,2000,8,18
2722093,0,1,0,2,NaN,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,291,4,1,3,2006,12,9
1727309,0,1,5,2,22.0,67.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,167,4,1,4,1990,9,23
48669,0,1,5,2,20.5,74.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,1,154,4,1,5,1997,9,12


In [79]:
df_val = pd.concat([y_val, X_val], axis=1)
df_val.head()

,Disq,Sex,Event,Equipment,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Tested,Federation,ParentFederation,Sanctioned,KgWeightClass,Year,Month,Day
2602872,0,1,5,5,NaN,55.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,246,24,1,3,2018,6,23
2470191,0,0,5,1,14.0,96.10,97.5,107.5,112.5,112.5,52.5,57.5,62.5,62.5,130.0,135.0,140.0,140.0,315.0,1,1,294,5,1,7,2024,2,17
2371880,1,1,0,1,48.0,122.38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,0,294,5,1,10,2015,2,21
1816457,1,1,5,2,NaN,81.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,296,4,1,6,1985,12,14
167214,0,1,5,1,26.0,104.34,270.0,285.0,297.5,297.5,177.5,190.0,0.0,190.0,330.0,355.0,375.0,375.0,862.5,3,1,74,4,1,8,2024,3,12


In [76]:
wr.s3.delete_objects(s3_training_file_location, boto3_session=boto3_session)
wr.s3.delete_objects(s3_validation_file_location, boto3_session=boto3_session)

In [80]:
wr.s3.to_csv(df=df_train,
            path=s3_training_file_location  + 'openpowerlift_train.csv',
            index=False, header=False, boto3_session=boto3_session
        )
wr.s3.to_csv(df=df_val,
            path=s3_validation_file_location  + 'openpowerlift_val.csv',
            index=False, header=False, boto3_session=boto3_session
        )

{'paths': ['s3://dante-sagemaker/openpowerlift/input/validation/openpowerlift_val.csv'],
 'partitions_values': {}}

In [82]:
# 테스트용 데이터는 파일로 저장
os.makedirs(f'dataset/{project_name}', exist_ok=True)
with open(f'dataset/{project_name}/test_dataset.pkl', 'wb') as f :
    pickle.dump((X_test, y_test), f)

XGBoost 모델 훈련

In [83]:
xgb_job_name = project_name + '-xgboost-training'
train_model_id, train_model_version = "xgboost", "1.7-1"

# https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/xgboost.html
xgb_container = sagemaker.image_uris.retrieve(
    "xgboost",
    sagemaker_session.boto_region_name,
    version="1.7-1",
    instance_type="ml.m5.xlarge",
)

xgb_estimator = sagemaker.estimator.Estimator(
    xgb_container,
    role,
    input_mode='File',
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=s3_xgb_output_location,
    sagemaker_session=sagemaker_session,
    checkpoint_s3_uri=s3_xgb_checkpoint_location,
    max_run=60 * 60,
    max_wait=60 * 60 * 2,
    use_spot_instances=True,
    base_job_name=xgb_job_name,
)

xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.1,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=1,
    objective='binary:logistic',
    num_round=20,
)

training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='text/csv')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='text/csv')

data_channels = {'train': training_input_config, 'validation': validation_input_config}

xgb_estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: openpowerlift-xgboost-training-2024-08-26-01-09-58-757


2024-08-26 01:09:59 Starting - Starting the training job......
2024-08-26 01:10:51 Downloading - Downloading input data...
2024-08-26 01:11:21 Downloading - Downloading the training image............
2024-08-26 01:13:18 Training - Training image download completed. Training in progress..[2024-08-26 01:13:28.686 ip-10-2-120-249.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-26 01:13:28.707 ip-10-2-120-249.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-08-26:01:13:29:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-26:01:13:29:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-08-26:01:13:29:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-26:01:13:29:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-08-26:01:13:29:INFO] Determined 0 GPU(s) available on the instance.
[2024-08-26:01:1

In [93]:
print('XGBoost 모델 아티팩트 위치 :', xgb_estimator.model_data)

XGBoost 모델 아티팩트 위치 : s3://dante-sagemaker/openpowerlift/output/xgboost/openpowerlift-xgboost-training-2024-08-26-01-09-58-757/output/model.tar.gz


XGBoost 모델 생성

In [94]:
sm_client = boto3_session.client('sagemaker')
xgboost_model_name = project_name + '-xgboost-inference-v1'

sm_client.create_model(
    ModelName=xgboost_model_name,
    PrimaryContainer={
        'Image': xgb_container,
        'ModelDataUrl': xgb_estimator.model_data
    },
    ExecutionRoleArn=role
)

{'ModelArn': 'arn:aws:sagemaker:ap-northeast-2:905418381372:model/openpowerlift-xgboost-inference-v1',
 'ResponseMetadata': {'RequestId': '9914b5f5-a47d-4bb3-a896-a425a92ada89',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9914b5f5-a47d-4bb3-a896-a425a92ada89',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Mon, 26 Aug 2024 01:26:04 GMT'},
  'RetryAttempts': 0}}

LightGBM 모델 훈련

https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_applying_machine_learning/lightgbm_catboost_tabtransformer_autogluon_churn/churn-prediction-lightgbm-catboost-tabtransformer-autogluon.ipynb

In [87]:
from sagemaker import hyperparameters

train_model_id, train_model_version, train_scope = "lightgbm-classification-model", "*", "training"
lgb_job_name = project_name + '-lightgbm-training'

# Retrieve the docker image
train_image_uri = sagemaker.image_uris.retrieve(
    region=None,
    framework=None,
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type="ml.m5.xlarge"
)

# Retrieve the training script
train_source_uri = sagemaker.script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)

train_model_uri = sagemaker.model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)


# Retrieve the default hyperparameters for training the model
default_hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)
default_hyperparameters.update({'num_boost_round': 120})

# Create SageMaker Estimator instance
lgbm_estimator = sagemaker.estimator.Estimator(
    role=role,  
    sagemaker_session=sagemaker_session,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1, # for distributed training, specify an instance_count greater than 1
    instance_type="ml.m5.xlarge",
    max_run=60 * 60, # 최대 학습 수행 시간
    max_wait=60 * 60 * 2, # 스팟인스턴스 사용시 자원 재확보를 위한 대기 시간
    use_spot_instances=True, # 스팟인스턴스 사용여부
    hyperparameters=default_hyperparameters, # 하이퍼파라터
    output_path=s3_lgb_output_location, # 학습 결과 저장 경로
    base_job_name=lgb_job_name
)

# Launch a SageMaker Training job by passing the S3 path of the training data
lgbm_estimator.fit(
    {
        "train": s3_training_file_location,
        "validation": s3_validation_file_location,
    }, logs=True, job_name=lgb_job_name
)

INFO:sagemaker:Creating training-job with name: openpowerlift-lightgbm-training


2024-08-26 01:15:29 Starting - Starting the training job...
2024-08-26 01:15:46 Starting - Preparing the instances for training...
2024-08-26 01:16:06 Downloading - Downloading input data...
2024-08-26 01:16:31 Downloading - Downloading the training image...
2024-08-26 01:17:22 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-08-26 01:17:27,397 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-08-26 01:17:27,398 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-26 01:17:27,407 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-08-26 01:17:27,409 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-08-26 01:17:28,134 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt

In [95]:
print('LightGBM 모델 아티팩트 위치 :', lgbm_estimator.model_data)

LightGBM 모델 아티팩트 위치 : s3://dante-sagemaker/openpowerlift/output/lightgbm/openpowerlift-lightgbm-training/output/model.tar.gz


LightGBM 모델 생성

In [96]:
inference_instance_type = "ml.m5.xlarge"
# LightGBM 이미지 URI 가져오기 (추론용)
deploy_image_uri = sagemaker.image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = sagemaker.script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

# LightGBM 모델 아티팩트 및 객체생성
lgbm_model_name = project_name + '-lightgbm-inference-v1'

sm_client.create_model(
    ModelName=lgbm_model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': deploy_image_uri,
        'ModelDataUrl': lgbm_estimator.model_data,
        'Environment': {
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_SUBMIT_DIRECTORY': deploy_source_uri
        }
    }
)

{'ModelArn': 'arn:aws:sagemaker:ap-northeast-2:905418381372:model/openpowerlift-lightgbm-inference-v1',
 'ResponseMetadata': {'RequestId': 'ba6560b6-c99e-43d9-ace7-58f1d5e9d2a9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ba6560b6-c99e-43d9-ace7-58f1d5e9d2a9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Mon, 26 Aug 2024 01:27:17 GMT'},
  'RetryAttempts': 0}}

### 실시간 엔드포인트 생성

멀티 Variants엔드포인트 생성

In [98]:
# 멀티 Variants 엔드포인트 생성

# 엔드포인트 및 구성 이름 설정
endpoint_name = name_from_base(project_name + '-' + 'xgb-multi-variants-endpoint')
endpoint_config_name = name_from_base(project_name + '-' + 'xgb-multi-variants-endpoint-config')

# 엔드포인트 설정 생성
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'XGBoost',
            'ModelName': xgboost_model_name,
            'InstanceType': inference_instance_type,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 0.5
        },
        {
            'VariantName': 'LightGBM',
            'ModelName': lgbm_model_name,
            'InstanceType': inference_instance_type,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 0.5
        }
    ]
)

# 엔드포인트 생성
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# 엔드포인트 생성 완료 대기
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

print(f"멀티 Variants 엔드포인트 '{endpoint_name}'가 생성되었습니다.")


멀티 Variants 엔드포인트 'openpowerlift-xgb-multi-variants-endpoi-2024-08-26-01-30-27-577'가 생성되었습니다.


- 콘솔에서도 진행할수 있습니다. (튜토리얼 참고)

### 실시간 AB테스트 엔드포인트 호출

AB테스트 엔드포인트 Variant 확인

In [99]:
# 엔드포인트의 variant 목록 조회
sm_client = boto3_session.client('sagemaker')

response = sm_client.describe_endpoint(EndpointName=endpoint_name)
production_variants = response['ProductionVariants']

print("엔드포인트의 variant 목록:")
for variant in production_variants:
    print(f"- Variant 이름: {variant['VariantName']}")
    print(f"  - 모델 이름: {variant['DeployedImages'][0]['SpecifiedImage']}")
    print(f"  - 현재 가중치: {variant['CurrentWeight']}")
    print(f"  - 현재 인스턴스수: {variant['CurrentInstanceCount']}")


엔드포인트의 variant 목록:
- Variant 이름: XGBoost-variant
  - 모델 이름: 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.7-1
  - 현재 가중치: 0.5
  - 현재 인스턴스수: 1
- Variant 이름: LightGBM-variant
  - 모델 이름: 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:2.0.1-cpu-py310
  - 현재 가중치: 0.5
  - 현재 인스턴스수: 1


In [100]:
# CSV 직렬화기 생성
serializer = sagemaker.serializers.CSVSerializer()

# 테스트 데이터의 처음 3개 행을 CSV 형식으로 직렬화
test_data = serializer.serialize(X_test.iloc[:3])

In [102]:
runtime_client = boto3_session.client("sagemaker-runtime")
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=test_data,
    TargetVariant='XGBoost'
    
)
print("XGBoost 예측 결과 :", list(map(float, response["Body"].read().decode('utf-8').strip().split('\n'))))


XGBoost 예측 결과 : [0.06557614356279373, 0.06558267027139664, 0.06557614356279373]


In [103]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=test_data,
    TargetVariant='LightGBM'
)
model_predictions = json.loads(response["Body"].read())
print("LightGBM 예측 결과 :", list(map(lambda x: x[1], model_predictions['probabilities'])))

LightGBM 예측 결과 : [0.036959384773807225, 0.036900182972338436, 0.025738902937078586]


* ABtest (무작위 호출)

In [104]:
from collections import defaultdict

In [109]:
y_preds = defaultdict(list)
y_trues = defaultdict(list)

In [110]:
# 100회 호출 및 결과 수집
for _ in range(100):
    # X_test에서 비복원 랜덤 추출로 1행 추출
    sample = X_test.sample(n=1)
    test_data = serializer.serialize(sample)
    
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',
        Body=test_data,
    )
    
    invoked_variant = response['InvokedProductionVariant']
    if invoked_variant == 'XGBoost':
        predictions = list(map(float, response["Body"].read().decode('utf-8').strip().split('\n')))
    elif invoked_variant == 'LightGBM':
        model_predictions = json.loads(response["Body"].read())
        predictions = list(map(lambda x: x[1], model_predictions['probabilities']))
    else:
        raise ValueError(f"알 수 없는 variant: {invoked_variant}")
    
    y_preds[invoked_variant].append(predictions[0]) # 1행만 추출했으므로 첫 번째 예측값만 저장
    y_trues[invoked_variant].append(y_test.loc[sample.index].values[0])

print(f"XGBoost 호출 횟수: {len(y_preds['XGBoost'])}")
print(f"LightGBM 호출 횟수: {len(y_preds['LightGBM'])}")

XGBoost 호출 횟수: 53
LightGBM 호출 횟수: 47


In [111]:
# 결과 분석
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

for variant, predictions in y_preds.items():
    print(f"\n* {variant} 결과 분석:")
    
    if not predictions:
        print(f"{variant}에 대한 호출 결과가 없습니다.")
        continue
    
    y_pred = [1 if p > 0.5 else 0 for p in predictions]
    
    accuracy = accuracy_score(y_trues[variant], y_pred)
    precision = precision_score(y_trues[variant], y_pred)
    recall = recall_score(y_trues[variant], y_pred)
    f1 = f1_score(y_trues[variant], y_pred)
    
    print(f"\t- 호출 횟수: {len(predictions)}")
    print(f"\t- 평균 예측값: {np.mean(predictions):.4f}")
    print(f"\t- 정확도: {accuracy:.4f}")
    print(f"\t- 정밀도: {precision:.4f}")
    print(f"\t- 재현율: {recall:.4f}")
    print(f"\t- F1 점수: {f1:.4f}")

# 두 variant 비교
print("\n* 두 모델 비교:")
v2_avg = np.mean(y_preds['XGBoost'])
v4_avg = np.mean(y_preds['LightGBM'])

print(f"\t- XGBoost 평균 예측값: {v2_avg:.4f}")
print(f"\t- LightGBM 평균 예측값: {v4_avg:.4f}")
print(f"\t- 예측값 차이 (v2 - v4): {v2_avg - v4_avg:.4f}")


* XGBoost-variant 결과 분석:
	- 호출 횟수: 53
	- 평균 예측값: 0.1630
	- 정확도: 1.0000
	- 정밀도: 1.0000
	- 재현율: 1.0000
	- F1 점수: 1.0000

* LightGBM-variant 결과 분석:
	- 호출 횟수: 47
	- 평균 예측값: 0.0921
	- 정확도: 1.0000
	- 정밀도: 1.0000
	- 재현율: 1.0000
	- F1 점수: 1.0000

* 두 모델 비교:
	- XGBoost 평균 예측값: 0.1630
	- LightGBM 평균 예측값: 0.0921
	- 예측값 차이 (v2 - v4): 0.0709


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix
import numpy as np

# 두 변형에 대한 혼동 행렬 계산
def get_confusion_matrix(y_true, y_pred):
    return confusion_matrix(y_true, [1 if p > 0.5 else 0 for p in y_pred])

cm_1 = get_confusion_matrix(y_trues['XGBoost'], y_preds['XGBoost'])
cm_2 = get_confusion_matrix(y_trues['LightGBM'], y_preds['LightGBM'])

# 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, title):
    return go.Heatmap(
        z=cm,
        x=['예측 음성', '예측 양성'],
        y=['실제 음성', '실제 양성'],
        colorscale='Blues',
        showscale=False,
        text=cm,
        texttemplate="%{text}",
        textfont={"size": 20},
    )

# 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=('XGBoost', 'LightGBM'))
fig.add_trace(plot_confusion_matrix(cm_1, 'XGBoost'), row=1, col=1)
fig.add_trace(plot_confusion_matrix(cm_2, 'LightGBM'), row=1, col=2)
fig.update_layout(
    title='XGBoost와 LightGBM의 혼동 행렬 비교',
    grid={'rows': 1, 'columns': 2, 'pattern': "independent"},
)
fig.show()


리소스 삭제

In [113]:
# 엔드포인트 삭제
print("엔드포인트 삭제 중...")
try:
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    print("엔드포인트가 성공적으로 삭제되었습니다.")
except Exception as e:
    print(f"엔드포인트 삭제 중 오류 발생: {e}")

# 엔드포인트 구성 삭제
print("엔드포인트 구성 삭제 중...")
try:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    print("엔드포인트 구성이 성공적으로 삭제되었습니다.")
except Exception as e:
    print(f"엔드포인트 구성 삭제 중 오류 발생: {e}")

# 모델 삭제
print("모델 삭제 중...")
for model_name in [xgboost_model_name, lgbm_model_name]:
    try:
        sm_client.delete_model(ModelName=model_name)
    except Exception as e:
        print(f"{model_name} 삭제 중 오류 발생: {e}")
print("모델이 성공적으로 삭제되었습니다.")

print("모든 리소스가 성공적으로 삭제되었습니다.")


엔드포인트 삭제 중...
엔드포인트가 성공적으로 삭제되었습니다.
엔드포인트 구성 삭제 중...
엔드포인트 구성이 성공적으로 삭제되었습니다.
모델 삭제 중...
모델이 성공적으로 삭제되었습니다.
모든 리소스가 성공적으로 삭제되었습니다.
